In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv(r'C:\Users\Ramya\Downloads\PlayTennis.csv')
print("\n Input Data Set is:\n", df)


 Input Data Set is:
      Outlook Temperature Humidity    Wind Play Tennis
0      Sunny         Hot     High    Weak          No
1      Sunny         Hot     High  Strong          No
2   Overcast         Hot     High    Weak         Yes
3       Rain        Mild     High    Weak         Yes
4       Rain        Cool   Normal    Weak         Yes
5       Rain        Cool   Normal  Strong          No
6   Overcast        Cool   Normal  Strong         Yes
7      Sunny        Mild     High    Weak          No
8      Sunny        Cool   Normal    Weak         Yes
9       Rain        Mild   Normal    Weak         Yes
10     Sunny        Mild   Normal  Strong         Yes
11  Overcast        Mild     High  Strong         Yes
12  Overcast         Hot   Normal    Weak         Yes
13      Rain        Mild     High  Strong          No


In [2]:
eps = np.finfo(float).eps
eps

2.220446049250313e-16

In [3]:
target=df.keys()[-1]
print('Target: ',target)
attributes=list(df.keys())
attributes.remove(target) 
print('Predicting Attributes: ', attributes)

Target:  Play Tennis
Predicting Attributes:  ['Outlook', 'Temperature', 'Humidity', 'Wind']


In [4]:
values_in_target = df[target].unique()
print(values_in_target)

['No' 'Yes']


In [5]:
def find_entropy_whole(df):
    overall_entropy = 0
    for value in values_in_target:
        p = df[target].value_counts()[value] / len(df[target])
        overall_entropy += -p * np.log2(p)
    return overall_entropy

find_entropy_whole(df)

0.9402859586706311

In [6]:
def find_entropy_of_attribute(df, attribute):
    values_in_attribute = df[attribute].unique()
    entropy_attribute = 0
    for value_in_attribute in values_in_attribute:
        overall_entropy = 0
        for value_in_target in values_in_target:
            num = len(df[attribute][df[attribute] == value_in_attribute][df[target] == value_in_target])
            den = len(df[attribute][df[attribute] == value_in_attribute])
            p = num / (den+eps)
            overall_entropy += -p * np.log2(p+eps)
        p2 = den / len(df)
        entropy_attribute += -p2 * overall_entropy
    return abs(entropy_attribute)

In [7]:
for attribute in df.keys()[:-1]:
    print(f'Entropy of the attribute "{attribute}" is :', find_entropy_of_attribute(df, attribute))

Entropy of the attribute "Outlook" is : 0.6935361388961914
Entropy of the attribute "Temperature" is : 0.9110633930116756
Entropy of the attribute "Humidity" is : 0.7884504573082889
Entropy of the attribute "Wind" is : 0.892158928262361


In [8]:
def find_best_attribute_to_divide(df):
    IG = []
    all_attributes = df.keys()[:-1]
    for attribute in all_attributes:
        IG.append(find_entropy_whole(df) - find_entropy_of_attribute(df, attribute))
    print(IG) 
    index_of_attribute_with_max_IG = np.argmax(IG)
    best_attribute = all_attributes[index_of_attribute_with_max_IG]
    return best_attribute
find_best_attribute_to_divide(df)

[0.24674981977443977, 0.029222565658955535, 0.15183550136234225, 0.048127030408270155]


'Outlook'

In [9]:
def buildTree(df, tree=None):
    node = find_best_attribute_to_divide(df)
    attValue = np.unique(df[node])
    if tree is None:
        tree = {}
        tree[node] = {}
    for value in attValue:
        subtable = df[df[node] == value].reset_index(drop=True)
        clValue, counts = np.unique(subtable[target], return_counts=True)
        if len(counts) == 1: #Checking purity of subset
            tree[node][value] = clValue[0]
        else:
            tree[node][value] = buildTree(subtable) # Calling the function recusively
    return tree

buildTree(df)

[0.24674981977443977, 0.029222565658955535, 0.15183550136234225, 0.048127030408270155]
[5.551115123125783e-16, 0.019973094021975557, 0.019973094021975557, 0.9709505944546682]
[5.551115123125783e-16, 0.5709505944546689, 0.9709505944546682, 0.019973094021975557]


{'Outlook': {'Overcast': 'Yes',
  'Rain': {'Wind': {'Strong': 'No', 'Weak': 'Yes'}},
  'Sunny': {'Humidity': {'High': 'No', 'Normal': 'Yes'}}}}

In [10]:
#https://stackoverflow.com/questions/13688410/dictionary-object-to-decision-tree-in-pydot
import pydot

data = buildTree(df)
def draw(parent_name, child_name):
    edge = pydot.Edge(parent_name, child_name)
    graph.add_edge(edge)

def visit(node, parent=None):
    for k,v in node.items():
        if isinstance(v, dict):
            # We start with the root node whose parent is None
            # we don't want to graph the None node
            if parent:
                draw(parent, k)
            visit(v, k)
        else:
            draw(parent, k)
            # drawing the label using a distinct name
            draw(k, k+'_'+v)

graph = pydot.Dot(graph_type='graph')
visit(data)
graph.write_png('output_graph.png')

[0.24674981977443977, 0.029222565658955535, 0.15183550136234225, 0.048127030408270155]
[5.551115123125783e-16, 0.019973094021975557, 0.019973094021975557, 0.9709505944546682]
[5.551115123125783e-16, 0.5709505944546689, 0.9709505944546682, 0.019973094021975557]
